![](figs/strathsdr_banner.png)

In [1]:
import os, warnings
from pynq import PL
from pynq import Overlay
from pynq import allocate
import numpy as np
from amc_cnn_overlay import OverlayAMCCNN
import scipy.io
import pickle
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.graph_objs as go

In [2]:
datapath = './test_data/testing_data_amc.pkl'
# load data
with open(datapath,'rb') as f:
    dataset = pickle.load(f)
classes = list(dataset.keys())

# Streaming-CNN FPGA Architecture for Communications-based Applications
----
This demonstration will present a modulation classification application for wireless communications modulation schemes running on a **AMD-Xilinx RFSoC 2x2 development board**.

## Modulation Classification
This demo intends to showcase the proposed streaming-CNN architecture running on an RFSoC 2x2 development board. Currently, the IP exists on-chip with inputs transferred via AXI4-Stream from the Processing System to Programmable Logic with future aims to connect directly to RF Data Converters on the RFSoC.

**Modulation Classification** is the task of indentifying what modulation scheme a received signal has been encoded with. The possible modulation schemes are: 
* 8PSK
* BPSK
* CPFSK
* GFSK
* PAM14
* QAM16
* QAM64
* QPSK

## Neural Network Structure

![](figs/NN.png)

----

## CNN Architecture
The CNN architecture has been built to support constant streaming inputs similar to how a classical communications pipeline is built with filters processing a stream of samples. This architecture aims to allow deep learning solutions to be inserted within an already existing communications pipeline. We assume samples from the air are constantly being received and this architecture has been built to support a stream without interruptions.

### Weight and Input Sample Restructure

To facilitate a streaming input convention, the order in which the neural network calculations are processed must be revisited. In a typical sliding window convolutional layer approach, the kernel weights may be processed over the input data multiple times. To simplify the calculations being performed on chip, the input data is transformed into a matrix equivalent of the sliding kernel approach. Similarly, the kernel weights are transformed into their matrix equivalent before deployment. The **left** figure below indicates how this is possible. The **right** figure shows how the input and kernel weights are transformed from 3D values to a matrix.

Convolutions to Matrix Multiplies  |  Transforming Inputs and Weights
:---------------------------------:|:-----------------------------------------:
![](figs/GEMMCalculations.png)             |  ![](figs/GEMM_inputs_weights.png)

### Overall Structure
The streaming-CNN architecture's overall structure is shown **below**. The architecture accepts a streaming input with one sample entering at a time before being stored in a 'Block RAM buffer'. The 'Read and Write Controller' performs the matrix conversion and passes columns of the resulting matrix to the 'Matrix-Vector Multiplier'. The resulting data is then passed through a 'ReLU' activation before it is ready of the next layer to be processed.

![](figs/overall_architecture.png)

<!-- #### Optimisations in Matrix-Vector Multipliers

Some of the matrix-vector multiplications can become quite large. We can take advantage of using a faster clock to time-share some of our resources. Below are two optimisations for both Convolutional and Dense neural network layers.

On the **left** there is a fully parallel Matrix-Vector Multiplier for when the resulting input vector is small enough to run calulations in parallel. On the **right** is the serial-parallel Matrix-Vector Multiplier where a subset of the multiple-accumulates are time-shared to reduce the resources used.

Fully Parallel Matrix-Vector Multiplier  |  Serial-Parallel Matrix-Vector Multiplier
:---------------------------------------:|:-----------------------------------------:
![](fullyparallel.png)                   |  ![](serial-parallel.png) -->

### Quantisation
The whole model runs with **18-bit fixed point** arithmetic for both inputs and weights. This value was chosen as it is the maximum fixed point length accepted by the DSP48s on-chip and maintains good precision of the original floating point weights.

----

# Interactive Demo
### Choose from 8 different modulation schemes and test out the CNN.
Firstly, the input waveform is plotted. Next the prediction by the model compared to the actual label.
Finally the prediction confidence by the CNN is displayed in the form of a bar graph.

In [3]:
from AMCLBWidget import AMCLBWidget

amc_widget=AMCLBWidget(dataset, classes)
amc_widget.display()

# Accuracy of CNN model
## Confusion matrix of the collated accuracies across all 8 modulation schemes.

In [ ]:
amc_widget.ol.clean()
mods = classes
predictions = {}
for mod in mods:
    predictions_mod = []
    data_mod = dataset[mod]
    for i in range(data_mod.shape[2]):
        data = data_mod[:,:,i]
        y = np.int16(data*np.int16(pow(2,14)))
        z = np.zeros(2*4096, dtype=np.int16)
        z[0::2] = y[0,:]
        z[1::2] = y[1,:]
        amc_widget.ol.send(z)
        amc_widget.ol.cnn.write(0x100,1)
        [y_pred,complex_data] = amc_widget.ol.receive()
        max_value = np.max(y_pred)
        max_index = np.where(y_pred == max_value)
        predictions_mod.append(max_index[0].tolist()[0])
        amc_widget.ol.clean()
    predictions[mod] = predictions_mod
    print(f"Done {mod}")
amc_widget.ol.clean()

In [ ]:
conf = np.zeros([len(mods),len(mods)])
confnorm = conf
for mod in mods:
    j = mods.index(mod)
    preds = predictions[mod]
    for i in range(len(preds)):
        k = preds[i]
        conf[j,k] += 1
for i in range(len(mods)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
confnorm_str = confnorm

In [ ]:
cm_text = [['','','','','','','',''],
           ['','','','','','','',''],
           ['','','','','','','',''],
           ['','','','','','','',''],
           ['','','','','','','',''],
           ['','','','','','','',''],
           ['','','','','','','',''],
           ['','','','','','','','']]
cm_diag = np.round(np.diag(confnorm_str*100.0),1)
for i in range(8):
    for j in range(8):
        if i == j:
            cm_text[i][j] = str(cm_diag[j])

In [ ]:
def plot_confusion_matrix(cm, title='',labels=[], text=cm_text):
    fig = go.Figure(go.Heatmap(z=cm, 
                               x=labels, 
                               y=labels,
                               colorscale=['#ffffff', '#023047'],
                               # colorscale=['#023047', '#219ebc', '#ffb703'],
                               # colorscale=['#fdfdfd','#000000'],
                               text=cm_text,
                               texttemplate="%{text}"))
    fig.update_layout(width=500, height=500, xaxis_title='true label', yaxis_title='predicted label', title=title)
    fig.update_layout(font=dict( 
        family='Latin Modern Math'
    ))
    fig.show()
    fig.write_image("modAccuracy.svg")

In [ ]:
plot_confusion_matrix(confnorm*100.0, labels=mods, text=cm_text)

----

----